In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

# import matplotlib
# import sklearn
import torch

from torch import nn, optim

from IPython.display import display
# plt.style.use("ggplot")

print("package版本信息：")
print("numpy:      ", np.__version__)
# print("pandas:     ", pd.__version__)
# print("matplotlib: ", matplotlib.__version__)
# print("sklearn:    ", sklearn.__version__)
print("PyTorch:     ", torch.__version__)

package版本信息：
numpy:       1.23.3
PyTorch:      1.12.0


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
# 项目本地数据集文件夹
LOCAL_DATA_PATH = r"D:\Project-Workspace\Python-Projects\DataAnalysis\local-datasets"

# Pytorch-Transformer

参考文档:
+ [Pytorch tutorial -> Transformer tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)
+ [Pytorch -> Transformer Layers](https://pytorch.org/docs/stable/nn.html#transformer-layers)

这里介绍了Pytorch里 transformer 层的实现模块.

主要有如下几个类：
+ `nn.Transformer`：一步到位构建整个Transformer模型，底层使用的是下面的API
+ `nn.TransformerEncoder`：Encoder层，包含了多个 `TransoformerEncoderLayer`
+ `nn.TransformerDecoder`：Decoder层，包含了多个 `TransformerDecoderLayer`
+ `nn.TransformerEncoderLayer`：单层Encoder
+ `nn.TransformerDecoderLayer`：单层Decoder

但是需要注意的是，上述各种层的**底部都是调用的`activation.py`中的`MultiheadAttention`类**。

## Multi-Head Attention Layer

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html#torch.nn.MultiheadAttention).

它是`TransformerEncoderLayer`和`TransformerDecoderLayer`层中 **self-attention 层**的实现。  
需要注意的是，pytorch源码中 `MultiheadAttention` 被放在了 Non-linear Activations 这一类中（并且是放在`activation.py`文件中的）。


### 实例化参数
`MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None)`  
+ `embed_dim`，整个MHA的输入/输出特征维度。  
  **原始论文里，MHA的输入/输出特征维度可以是不一样的，但是在Pytorch的实现里，强制要求它们是一样的**（这个限制未来版本可能会取消）。
+ `num_heads`，head数量，注意，分配到每个head的维度 = embed_dim/num_heads.
+ `kdim`，自定义 key 中的特征维度，它是下面`forward()`方法里`key`对应的特征维度，默认下=`embed_dim`
+ `vidm`，自定义 value 中的特征维度，`forward()`方法里`value`对应的特征维度，默认=`embed_dim`
+ `bias`，
+ `add_bias_kv`
+ `add_zero_attn`
+ `batch_first`，早期版本的pytorch中，会要求下面`forward()`方法里输入数据的各个维度顺序必须为：`(序列长度，batch_size, embedding_dimension)`，后来提供了这个`batch_first`参数，用于适应一下输入数据的维度变化.


### 前向传播

`MultiheadAttention.forward(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None)`
+ `query`, `key`, `value`，MHA的三个输入矩阵——这三个输入向量会被用于生成对应的Q,K,V
  + `query.shape` = $(L, N, E_q)$，`key.shape` = $(S, N, E_k)$，`value.shape` = $(S, N, E_v)$.   
    + $N$ 是 batch size
    + $E$ 是 embedding 维度，$E_q$=`embed_dim`, $E_k$=`kdim`, $E_v$=`vdim`
    + $L$ 是**target**序列的长度——因为它决定了MHA输出的序列长度
    + $S$ 是**source**序列的长度
  + Transformer中MHA会**在3个地方使用，这3个地方会影响 query, key, value 的内容**
    + Encoder中 query, key, value 都是来自于样本的**输入序列**，此时 $L=S$，`kdim == vdim`
    + Decoder的 **Masked MHA** 中 query, key, value 都来自于样本的**输出序列**，此时 $L=S$, `kdim == vdim`
    + Decoder的 **decoder-encoder attention**中，query 来自于样本的**输出序列**，key, value 都是**Encoder最后一层的输出**，此时可能 $L\neq S$, `kdim != vdim`
  + **有关序列维度的说明**
    + 输入的 `query`, `key`, `value` 不是self-attention的输入，它们需要经过各自的投影矩阵 $W_Q \in (E_q, E_q)$, $W_K \in (E_d, E_k)$, $W_V \in (E_d, E_v)$ 的转换，上面这3个参数矩阵的维度，就是MHA里初始化的维度
    + 线性投影过程如下（这里把batch的维度提前了）：   
    $q = query * W_Q^T \in (N, L, E_d)$,   
    $k = key * W_K^T \in (N, S, E_d)$,   
    $v = value * W_V^T \in (N, S, E_d)$  
    + 可以看出，参数的 `kdim` 和 `vdim` 只是用于设置投影矩阵 $W_K, W_V$ 的维度，这两个矩阵的另一个维度被设置成了 $E_q$=`embed_dim`，经过上述三个投影矩阵的变换之后，`query`, `key`, `value` 的维度都变成了 $E_q$，也就是 `embed_dim`
    + 后续进行的MHA操作为：
    $${\rm softmax}(\frac{q*k^T}{\sqrt{E_d}}) * v$$
    $$((N, L, E_d) * (N, E_d, S)) * (N, S, E_d) \rightarrow (N, L, E_d)$$
    调用的是`torch.bmm()`算子，会在每个批次上做上述操作。
    + 可以看出，最终决定输出序列长度的是 `query` 的序列长度 $L$，而 `key` 和 `value` 的序列长度必须一致，它们会在中间计算过程中相互抵消掉。


+ `key_padding_mask`：填充位置的掩码，shape=$(N, S)$   
  + 对于一个batch的训练数据来说，batch size 为 $N$ 表示有 $N$ 个序列，但是**每个序列的长度（也就是单词个数）是不一样的**，`key`,`value`中的 $S$ 都是指最长序列的长度，其他的序列就需要对缺少的单词位置做填充(padding)，padding位置的词通常使用全为0的embeding 来表示。   
  + 为了不让MHA对这些位置的词产生注意力，需要记录这些padding位置，每个batch都使用这样一个 $(N, S)$ 的矩阵来标识padding的位置，该矩阵的每一行对应batch中的一个序列，值为`[False, False, True, True]` 或者 `[0, 0, 1, 1]` 的形式，其中结尾`True`或者1标识的就是padding的位置。   
  + 需要注意的是，这里padding矩阵实际生效位置是在`query`和`key`做完矩阵乘法得到shape=$(N,L,S)$的中间矩阵 P 之后才使用的，由于只有`key`使用了padding掩码，只需要在 $N$ 个序列中对 $S$ 这一个维度标识padding位置即可，所以使用的padding矩阵shape=$(N,S)$. pytorch中的实现就是对中间矩阵 P 的 $S$ 维度，按照 `[False,False,True,True]` 中的标记，对应`True`的位置赋值为`-inf`，从而在后续的softmax中，该位置得到的概率为0.


+ `attn_mask`，**只在Decoder中使用**，主要是解码时不让当前位置的词和后面要预测的词序列进行Attention.   
  + shape=$(L,S)$ 或者 $(N * num\_heads,L,S)$，如果是 2D 的矩阵，那么batch中的每个序列都会使用这个，如果是 3D 的矩阵，则会指定每个batch 的掩码
  + 它的生效位置也是在`query`和`key`相乘得到shape=$(N,L,S)$的中间矩阵 P 之后使用，此时batch中**每个**序列的输入`query`部分（shape=$(L,E)$）和`key`部分（shape=$(S,E)$）相乘得到了一个 $(L,S)$ 的矩阵 P，矩阵中列表示`query`里的每个词，行表示`key`中的每个词，如果不想让`query`中第 $i$ 个词看到`key`中第 $j$ 个词之后的内容，只需要让 `P[i, j:]` 这部分为 `-inf` 即可，因此这个掩码的shape=$(N,L,S)$.
  + 如果是在Decoder中的**Masked MHA**这里使用，此时输入的`query`,`key`,`value`都是来源于同一个地方，`query`和`key`维度相等，此时 P 是一个方阵，并且右上三角均为`-inf` —— 这是好理解的部分，也是正常它要解决的问题部分。
  + 如果是在Decoder中的**encoder-decoder attention**层使用，此时输入的`query`是来自于之前的Masked MHA层，属于输出序列的内容，但是`key`,`value`是来自于Encoder，属于输入序列的内容，此时才会出现`query`和`key`长度不等的情况——这种情况下虽然也可以使用掩码，但是它的含义就不那么好解释了。


+ `need_weights`，bool，表示是否需要返回`attn_output_weigths`，默认为`True`


+ `forward()`方法返回值
  + `attn_output`，shape= $(L, N, E_q)$，$L$ 由 输入的`query`序列长度决定，$E$=`embed_dim`.
  + `attn_output_weights`，shape=$(N, L, S)$，这个权重应该是每个输入序列中，每个word的attention权重.
  

### 一些说明

Pytorch中的Multi-Head Attention的实现和论文《All you need is transformer》中有一些不一样的地方，总结如下：

+ pytorch在实现MHA时，强制要求了输入的特征维度和输出的特征维度必须一致，也就是实例化时的`embed_dim`参数，但是在原论文中没有这个限制。   
原论文中，输入的特征维度是`query`里的embed_dim，输出的特征维度由`value`的embed_dim决定。

+ pytorch中MHA实例化参数 `kdim`,`vdim` 和含义和原论文里的 $d_k$, $d_v$ 不一样，如下所示.   
原论文中的 $d_k$, $d_v$ 指的是 query, key 经过参数矩阵 $W$ 变换之后的 embed_dim，而pytoch中的 `kdim`,`vdim` 指定的是输入的 `key`,`value` 中，没有经过 $W$ 变换之前的特征维度，$W$ 这个参数矩阵设置的时候，其中一个维度就是 `kdim` （或者 `vdim`），另一个维度就是 `embed_dim`，因此 `key`, `value` 经过 $W$ 变换之后，维度就变成了 `embed_dim` ——这是 pytorch强制规定的。

<img src="images/multi-head-attention-formula.png" width="60%" align="left">

### 使用示例

In [2]:
embed_dim, num_heads = 512, 8
batch_size, target_len, source_len = 5, 10, 20

query = torch.rand(target_len, batch_size, embed_dim)
key = torch.rand(source_len, batch_size, embed_dim)
value = torch.rand(source_len, batch_size, embed_dim)

In [3]:
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
attn_output, attn_output_weights = multihead_attn(query, key, value)
print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


In [4]:
kdim, vdim = 256, 256
key = torch.rand(source_len, batch_size, kdim)
value = torch.rand(source_len, batch_size, vdim)

multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, kdim=kdim, vdim=vdim)
attn_output, attn_output_weights = multihead_attn(query, key, value)

print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


## Encoder Layer

### 单层Encoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html#torch.nn.TransformerEncoderLayer)，它内部包含了一个 self-attention 层 + 一个 feedforward 层.

`nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`  

+ 实例化参数
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1  


+ 前向传播`forward(src, src_mask=None, src_key_padding_mask=None)`
  + `src`，输入的sequence, `shape` = $(S, N, E)$
  + `src_mask`，输入sequence的mask
  + `src_key_padding_mask`，每个batch中src的padding矩阵  


+ `forward()`方法返回值  
  它返回的是和 `src` shape 相同的 tensor.


Encoder的内部只有**一层**`MultiheadAttention`：  
`Encoder.forward()`实际执行时，调用方式为`MultiheadAttention.forward(src, src, src)`——`src`会同时作为 query, key, value 传入 self-attention。



### 多层Encoder

`TransformerEncoder(encoder_layer, num_layers, norm=None)`
+ 参数：
  + `encoder_layer`，是一个`TransformerEncoderLayer`类的实例对象
  + `num_layers`，指定堆叠的 Encoder 层数
+ `forward()`方法和`TransformerEncoderLayer`的一致

In [5]:
d_model, num_heads, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
src = torch.rand(source_len, batch_size, d_model)

encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=dim_ffn)
out = encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


In [8]:
multi_encoder_layer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=3)
out = multi_encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


## Decoder Layer

### 单层Decoder

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.TransformerDecoderLayer.html#torch.nn.TransformerDecoderLayer)

`TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`   

+ 实例化参数：和 Encoder 一致
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  + `dropout`，dropout比例，默认 0.1


+ 前向传播`forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`  
  这个方法和 Encoder 不一样，它多了一个 memory. 
  + `tgt`，输入的序列，`shape`= $(S, N, E)$，
  + `memory`，来自 encoder 层（通常是最后一层）的输出, `shape`=$(S, N, E)$.
  + `tgt_mask`，
  + `memory_mask`
  + `tgt_key_padding_mask`，
  + `memory_key_padding_mask`

> `Decoder`内部会调用两层`MultiheadAttention`
> 1. 第一层调用时为`MultiheadAttention.forward(tgt, tgt, tgt)`
> 2. 第二层调用时为`MultiheadAttention.forward(tgt, memory, memory)`


+ `forward()`方法返回值  
它返回的是和 `tgt` shape 相同的 tensor，


### 多层Decoder

`TransformerDecoder(decoder_layer, num_layers, norm=None)`   
参数同 `TransformerEncoder` 类.

In [7]:
d_model, nhead, dim_ffn = 512, 8, 2048
batch_size, source_len = 10, 32
tgt = torch.rand(source_len, batch_size, d_model)
memory = torch.rand(source_len, batch_size, d_model)

decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_ffn)
decoder_out = decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


In [9]:
multi_decoder_layer = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=3)
decoder_out = multi_decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


## Transformer类

[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#torch.nn.Transformer)

`nn.Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1, activation=<function relu>, custom_encoder=None, custom_decoder=None, layer_norm_eps=1e-05, batch_first=False, norm_first=False, device=None, dtype=None)`

+ 实例化参数
  + `d_model`，输入/输出的embedding dim，默认512
  + `nhead`
  + `num_encoder_layers`
  + `num_decoder_layers`
  + `dim_feedforward`
  + `dropout`
  + `activation`
  
+ 正向传播`forward(src, tgt, src_mask=None, tgt_mask=None, memory_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`
  + `src`，shape=$(S,N,E)$
  + `tgt`，shape=$(T,N,E)$
  + `src_mask`，shape=$(S,S)$
  + `tgt_mask`，shape=$(T,T)$
  + `memory_mask`，shape=$(T,S)$
  + `src_key_padding_mask`，shape=$(N,S)$
  + `tgt_key_padding_mask`，shape=$(N,T)$
  + `memory_key_padding_mask`，shape=$(N,S)$

----------------------------

# Huggingface

[Huggingace公司](https://huggingface.co/docs)（下面简称HF）提供了围绕Transformer架构的一系列工具，我常用的有如下几个部分：
+ [Datasets](https://huggingface.co/docs/datasets/index)，提供了一系列的数据集封装，方便下载常用的训练数据，同时也封装了一些数据预处理操作
+ [Tokenizer](https://huggingface.co/docs/tokenizers/index)，为NLP提供了分词处理的pipeline，底层是由Rust实现的，分词速度非常快，适用于生产环境
+ [Transformer](https://huggingface.co/docs/transformers/index)，基于transformer架构实现的一系列模型，比如BERT等


此外，HF还提供了许多详细的文档资源：
+ **[Course](https://huggingface.co/course/chapter0/1?fw=pt) ——KEY**，HF提供的一个教程，详细介绍了如何使用上述HF-ecosystem的各种package，这个很有用
+ [Models](https://huggingface.co/models)，模型仓库，提供了一系列任务的预训练模型，可以配合`transformer`包使用
+ [Dataset](https://huggingface.co/datasets)，数据集仓库，提供许多用于训练的数据集，可以配合`datasets`包使用


下面是对上述我常用的3个包的整理总结.

----

## Dataset

`datasets`包可以方便的下载常用的数据集，并且封装了一些数据处理操作.

详细介绍可以参考：
+ github仓库 [datasets](https://github.com/huggingface/datasets)
+ 官方文档 [Datasets](https://huggingface.co/docs/datasets/index)
+ 官方教程文档 [Course --> The datasets library](https://huggingface.co/course/chapter5/1?fw=pt)：这个作为入门介绍比较容易接受
 
huggingface官方提供了一个数据集的托管共享平台：[datasets hub](https://huggingface.co/datasets).

### dataset介绍

这部分的内容主要来自于官方文档 [datasets: Conceptual Guides](https://huggingface.co/docs/datasets/about_arrow).

**一个dataset对应于一个文件夹**（[Conceptual Guides --> Build and load](https://huggingface.co/docs/datasets/about_dataset_load)），其中主要包含了下面两部分内容：
1. 数据集本身，通常以JSON，csv，parquet等格式存储
2. 处理数据集的Python脚本——**可选** 

一般datasets-hub上提供的数据集，基本都有脚本，**脚本中会从URL下载原始数据集，进行一些数据处理，然后生成便于使用的数据集格式（Arrow格式）**，并且此脚本会返回一个`DatasetBuilder`对象，记录数据集的元数据信息。

datasets中提供的数据一般会以Apache-Arrow的格式存储，这是一种常用于存储大数据的列存储格式。

### 数据集加载

参考文档 [datasets: Tutorials --> Load a dataset from the Hub](https://huggingface.co/docs/datasets/load_hub#load-a-dataset-from-the-hub).

`datasets` 模块提供了如下的函数用于加载数据：

+ `list_datasets()`，以list的形式列出可用的数据集（实际上是处理数据集的脚本），不过**推荐直接去 dataset-hub 里查找**，因为数据集实在是太多了，这个返回结果会很大，很卡。

+ `load_dataset_builder()`，加载数据集的builder
  + 内部实例化一个`dataset_module_factory`对象，**会触发下载数据集中的Python脚本和元数据的操作，但不会去下载实际的数据集**
  + 返回对应的`DatasetBuilder`对象，记录数据集的元数据信息
  + **这个是最基础的函数，下面的大部分函数都会先调用这个**


+ `get_dataset_config_info()`，加载数据集的配置信息
  + 内部调用`load_dataset_builder()`
  + 返回`DatasetBuilder`对象的`.info`属性   


+ `get_dataset_config_names()`，返回数据集的配置信息
  + 有些数据集会有**多个类型的子数据集(sub-datasets)，**这些子数据集被称为 *configurations*，可以用这个函数查看指定数据集含有哪些子数据集
  + 内部也会实例化一个`dataset_module_factory`对象    


+ `get_dataset_infos()`，查看数据集信息.  
  + 内部调用`get_dataset_config_names()`
  + 返回一个dict         


+ `get_dataset_split_names()`，获取数据集中split信息，**一个split就是数据集的子集，比如训练集，测试集等**
  + 内部调用`get_dataset_config_info()`         


+ **`load_dataset()`——KEY**，最重要的函数，内部会执行如下几件事（参见[接口文档](https://huggingface.co/docs/datasets/package_reference/loading_methods#datasets.load_dataset)）：
  1. 调用`load_dataset_builder()`获取数据集的元数据和Python脚本
  2. 从Python脚本中的URL下载原始数据集，并进行处理             



`load_dataset()`接受的参数如下（大部分参数也可用于`load_dataset_builder()`）
+ `path`：数据集的path或者name，有如下几种情况：
  + 本地路径：
    + 文件夹中只有数据文件，则基于文件类型(csv,txt,json等)使用通用的builder，包含文件夹下的所有的数据
    + 指向从huggingface-hub中已经下载好的对应数据集的处理脚本，则会运行该脚本，创建对应的builder对象
  + 数据集名称：
    + 指向huggingface-hub中的数据名称，然后按照上面的方式处理，如果hub上的该数据只有数据，就使用通用的builder，如果含有处理脚本，就使用对应的处理脚本
    
  注意，**如果是指向数据处理脚本，就不会再下载数据集的元数据信息了**
  
+ `name`：指定 data configuration 的名称
+ `data_dir`：指向 data configuration 的位置
+ `data_files`：指向数据文件的位置
+ `cache_dir`：缓存位置，默认为 `~/.cache/huggingface/datasets`

+ `split`：获取数据集的哪个子集
  + 不指定时，返回该数据集的所有split，封装成一个`DatasetDict`
  + 指定时，返回的是`Dataset`


In [32]:
from datasets import load_dataset_builder, get_dataset_config_names, get_dataset_config_info, \
    get_dataset_infos, get_dataset_split_names,  load_dataset, Dataset

In [3]:
# 返回的 list 太长了，其实没啥用
# datasets_list = list_datasets()
# print(len(datasets_list))
# datasets_list[:5]

#### 加载在线数据集

这里以 wikitext 这个数据集为例，该数据集包含多个类型的子数据集。

In [8]:
# 如果只使用如下的数据集名称，那么第一次会从 huggingface-hub 下载该数据集的元数据信息和数据处理脚本（wikitext.py）
# 后续操作虽然不用再次下载数据处理脚本，但是也会联网，速度比较慢
# path = 'wikitext'

# 推荐的操作是，下载了数据处理脚本之后，path 直接指定本地的数据脚本，这样后续的各种操作会快很多
path = os.path.join(LOCAL_DATA_PATH, r'huggingface\wikitext.py')
print(os.path.exists(path))

True


In [9]:
# 不要一开始就直接查看 builder，会报错，因为该数据集中有多个子数据集配置，所以还需要指定子数据集的配置
# builder = load_dataset_builder(path)

# 先查看该数据集有哪些配置
# 这里可能会下载数据集的处理脚本和元数据信息
get_dataset_config_names(path)

['wikitext-103-v1',
 'wikitext-2-v1',
 'wikitext-103-raw-v1',
 'wikitext-2-raw-v1']

In [11]:
# 也可以使用下面这个方法查看数据集信息，但是返回的内容比较多，速度也比较慢
info = get_dataset_infos(path)
print(info)

{'wikitext-103-v1': DatasetInfo(description=' The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified\n Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike\n License.\n', citation='@misc{merity2016pointer,\n      title={Pointer Sentinel Mixture Models},\n      author={Stephen Merity and Caiming Xiong and James Bradbury and Richard Socher},\n      year={2016},\n      eprint={1609.07843},\n      archivePrefix={arXiv},\n      primaryClass={cs.CL}\n}\n', homepage='https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/', license='Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)', features={'text': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='wikitext', config_name='wikitext-103-v1', version=1.0.0, splits={'test': SplitInfo(name='test', num_bytes=129

In [10]:
# 查看数据有哪些分割子集，必须要带上配置信息，也就是具体的子数据集
get_dataset_split_names(path, 'wikitext-2-raw-v1')

['test', 'train', 'validation']

In [14]:
# 加载数据集的 Builder，这里也要带上配置信息
builder = load_dataset_builder(path, name='wikitext-2-raw-v1')
print('builder.__class__: ', type(builder))

builder.__class__:  <class 'datasets_modules.datasets.wikitext.a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126.wikitext.Wikitext'>


In [15]:
# 查看 Builder 的各种属性
print('builder.name:')
print(builder.name)
print('----------------')
print('builder.config_id:')
print(builder.config_id)
print('----------------')
print('builder.config:')
print(builder.config)
print('----------------')
print('builder.cache_dir:')
print(builder.cache_dir)

builder.name:
wikitext
----------------
builder.config_id:
wikitext-2-raw-v1
----------------
builder.config:
WikitextConfig(name='wikitext-2-raw-v1', version=1.0.0, data_dir=None, data_files=None, description='Raw level dataset: the raw tokens before the addition of <unk> tokens. They should only be used for character level work or for creating newly derived datasets.')
----------------
builder.cache_dir:
C:\Users\Daniel-ZHANG\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126


In [16]:
# 查看 Builder 的 info信息
print("builder.info.__class__: ")
print(builder.info.__class__)
print('-----------------------')

# 查看数据集描述信息
print('builder.info.builder_name:')
print(builder.info.builder_name)
print('-----------------------')
print('builder.info.config_name:')
print(builder.info.config_name)
print('-----------------------')
print('builder.info.description:')
print(builder.info.description)
print('-----------------------')
print('builder.info.features:')
print(builder.info.features)
print('-----------------------')
print('builder.info.dataset_size:')
print(builder.info.dataset_size)
print('-----------------------')
print('builder.info.download_size:')
print(builder.info.download_size)
print('-----------------------')
print('builder.info.splits:')
builder.info.splits

builder.info.__class__: 
<class 'datasets.info.DatasetInfo'>
-----------------------
builder.info.builder_name:
wikitext
-----------------------
builder.info.config_name:
wikitext-2-raw-v1
-----------------------
builder.info.description:
 The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified
 Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike
 License.

-----------------------
builder.info.features:
{'text': Value(dtype='string', id=None)}
-----------------------
builder.info.dataset_size:
13526093
-----------------------
builder.info.download_size:
4721645
-----------------------
builder.info.splits:


{'test': SplitInfo(name='test', num_bytes=1305088, num_examples=4358, dataset_name='wikitext'),
 'train': SplitInfo(name='train', num_bytes=11061717, num_examples=36718, dataset_name='wikitext'),
 'validation': SplitInfo(name='validation', num_bytes=1159288, num_examples=3760, dataset_name='wikitext')}

In [20]:
# 不指定数据集的 split 时，会加载所有的 split，返回 DatasetDict
# 如果 cache_dir 里没有数据，那么这里第一次会下载数据集
data = load_dataset(path, 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
data = load_dataset(path, 'wikitext-2-raw-v1')

Found cached dataset wikitext (C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
print("data.__class__: ", data.__class__)
data

data.__class__:  <class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [19]:
# 指定加载数据集的某个split
data_train = load_dataset(path, 'wikitext-2-raw-v1', split='train')
data_train

Found cached dataset wikitext (C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


Dataset({
    features: ['text'],
    num_rows: 36718
})

In [42]:
data_train.cache_files

[{'filename': 'C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\\wikitext-train.arrow'}]

#### 加载本地自定义数据

In [21]:
custom_data_path = os.path.join(LOCAL_DATA_PATH, 'rankingcard.csv')
print(os.path.exists(custom_data_path))

True


In [24]:
import pandas as pd
df = pd.read_csv(custom_data_path)
df.head()

,index,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [29]:
# 下面会在 custom_data_path 路径下自动创建一个同名的文件夹
ds = load_dataset('csv', data_files=custom_data_path)

Using custom data configuration default-30075934ae74f7bd
Found cached dataset csv (C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/csv/default-30075934ae74f7bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
ds

DatasetDict({
    train: Dataset({
        features: ['index', 'SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents'],
        num_rows: 150000
    })
})

#### 手动创建

指的是从pandas.DataFrame或者dict对象中手动创建。

此时可以调用`Dataset`类的如下类方法：
+ `from_dataframe()`
+ `from_dict()`
+ `from_generator()`
+ `from_parquet()`
+ `from_list()`
+ ...

In [31]:
data_dict = {
    'col-1': [1, 2, 3],
    'col-2': ['a', 'b', 'c']
}
data_df = pd.DataFrame(data_dict)

In [35]:
ds1 = Dataset.from_dict(data_dict)
ds1

Dataset({
    features: ['col-1', 'col-2'],
    num_rows: 3
})

In [36]:
ds2 = Dataset.from_pandas(data_df)
ds2

Dataset({
    features: ['col-1', 'col-2'],
    num_rows: 3
})

In [40]:
ds1.data

InMemoryTable
col-1: int64
col-2: string
----
col-1: [[1,2,3]]
col-2: [["a","b","c"]]

In [41]:
ds1.cache_files

[]

### 数据集处理


上面加载数据集后，返回的 `Dataset` 对象，提供了一系列操作数据和处理数据的方法。

In [20]:
path = os.path.join(LOCAL_DATA_PATH, r'huggingface\wikitext.py')
builder = load_dataset_builder(path, name='wikitext-2-raw-v1')
data_train = load_dataset(path, 'wikitext-2-raw-v1', split='train')
data_train

Found cached dataset wikitext (C:/Users/Daniel-ZHANG/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


Dataset({
    features: ['text'],
    num_rows: 36718
})

In [23]:
data_train.__class__

datasets.arrow_dataset.Dataset

In [8]:
data_train.num_rows

36718

In [9]:
data_train.num_columns

1

In [13]:
data_train.shape

(36718, 1)

In [12]:
data_train.column_names

['text']

In [11]:
data_train.data.__class__

datasets.table.MemoryMappedTable

In [16]:
item = data_train[0]
print(item.__class__)

<class 'dict'>


#### 数据集切片、选取

In [77]:
# 获取第 1 个样本
data_train[0]

{'text': ''}

In [83]:
# 获取 text 特征
text = data_train['text']
print(text.__class__)
print(len(text))

<class 'list'>
36718


In [85]:
text[:5]

['',
 ' = Valkyria Chronicles III = \n',
 '',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more f

In [88]:
# 直接切片
data_train[:5]

{'text': ['',
  ' = Valkyria Chronicles III = \n',
  '',
  ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
  " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making th

#### Map操作

支持以自定义函数 按行 或者 按batch 的方式，处理数据集.

In [7]:
data_train[0]

{'text': ''}

+ 按行 处理

In [8]:
# 只取每个样本的前10个字符
def example_head(example):
    example['head'] = example['text'][:5]
    return example

ds_1 = data_train.map(example_head)

  0%|          | 0/36718 [00:00<?, ?ex/s]

In [9]:
ds_1.__class__

datasets.arrow_dataset.Dataset

In [12]:
ds_1[:3]

{'text': ['', ' = Valkyria Chronicles III = \n', ''],
 'head': ['', ' = Va', '']}

+ 按batch 处理

In [58]:
def example_chunk(examples):
    # examples 现在是一个 batch 的数据
    print(examples.__class__)
    print(len(examples['text']))
    # print(examples)
    # print(examples['text'])
    # chunks 的长度必须要和 examples 一致，因为 chunks 中元素对应于 examples 中的每个样本
    batch_size = len(examples['text'])
    # 这里只是简单的记录每个 batch 中各个样本的长度
    chunks = [[len(example) for example in examples['text']] for _ in range(batch_size)]
    # print(chunks)
    print('-------------------------------------------')
    return {"chunks": chunks}

ds_2 = data_train.select([0,1,2,3,4,5]).map(example_chunk, batched=True, batch_size=3)

  0%|          | 0/2 [00:00<?, ?ba/s]

<class 'datasets.arrow_dataset.Batch'>
3
-------------------------------------------
<class 'datasets.arrow_dataset.Batch'>
3
-------------------------------------------


In [59]:
ds_2.__class__

datasets.arrow_dataset.Dataset

In [60]:
len(ds_2)

6

In [61]:
ds_2[0]

{'text': '', 'chunks': [0, 30, 0]}

In [62]:
ds_2[1]

{'text': ' = Valkyria Chronicles III = \n', 'chunks': [0, 30, 0]}

In [63]:
ds_2[3]

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 'chunks': [706, 524, 574]}

-----

## Tokenizer

官方参考文档： 
+ [Tokenizer](https://huggingface.co/docs/tokenizers/index)
+ [Course --> 6. The Tokenizer Library](https://huggingface.co/course/chapter6/1?fw=pt)


一个tokenizer里包含了如下4个步骤（[Tokenizer -> The Tokenization pipeline](https://huggingface.co/docs/tokenizers/pipeline#the-tokenization-pipeline)）：
1. **Normalization**：单词正则化，比如大小写处理，词形还原等
2. **Pre-tokenization**：对单词进行简单的分词处理
3. **Model**：使用更加细致的分词模型，比如WordPiece等
4. **Postprocessor**：对分词后的序列进行后处理，添加BERT模型所需要的特殊token，比如`[CLS]`,`[SEP]`,`[PAD]`等

上述4个步骤对应于`tokenizer`包里的4个模块，也是通过这4个模块来组成一个pipeline的，每个模块常用的选择可以参考官方文档 [Tokenizer -> Components](https://huggingface.co/docs/tokenizers/components#components)。

此外，通常 tokenizer 还提供了一个 **Decoding** 步骤，用于将上面得到的词的id转换成原有的text。


注意：
> `transformer`包中各种模型使用的tokenizer的fast实现，就依赖于`tokenizer`包，但是slow版本的实现并不依赖。

### 分词模型

参考文档：
+ [Course --> 6. The tokenizers library -> Normalization and pre-tokenization](https://huggingface.co/course/chapter6/4?fw=pt#normalization-and-pre-tokenization)
+ [Course --> 6. The tokenizers library -> Byte-Pair Encoding tokenization](https://huggingface.co/course/chapter6/5?fw=pt#byte-pair-encoding-tokenization)
+ [Course --> 6. The tokenizers library -> WordPiece tokenization](https://huggingface.co/course/chapter6/6?fw=pt#wordpiece-tokenization)
+ [Course --> 6. The tokenizers library -> Unigram tokenization](https://huggingface.co/course/chapter6/7?fw=pt#unigram-tokenization)

主要分为如下几类：
+ Byte-Pair Encoding(BPE)
+ WordPiece
+ Unigram

### tokenizer使用

`Tokenizer`类的属性如下：
+ `normalizer`属性，pipeline中使用的`Normalizer`类
+ `pre_tokenizer`属性，pipeline中使用的`PreTokenizer`类
+ `model`属性，...
+ `post_processor`属性，...
+ `decoder`属性，....
+ `padding`属性，返回一个dict，记录当前的padding配置
+ `truncation`属性，返回一个dict，记录当前的截断配置

`Tokenizer`类的常用方法如下：
+ `from_pretrained()`，加载已训练好的token
+ `encode(sequence, pair = None, is_pretokenized = False, add_special_tokens = True)`：将word转成token id
+ `encode_batch()`：批量转换
+ `decode(ids, skip_special_tokens = True)`：将token id 转回 word
+ `decode_batch()`：批量转换
+ `token_to_id()`：
+ `id_to_token()`：
+ `train(files, trainer = None )`：训练分词器
+ `train_from_iterator(iterator, trainer = None )`：训练分词器
+ `save()`：保存分词器

上面`encode()`方法返回的是`Encoding`对象，该对象是dict的子类，有如下属性和方法：
+ `ids`：每个token的ID
+ `tokens`：具体的token
+ `attention_mask`：用于告诉Bert等模型，哪些位置（值为1）的词需要进行attention操作
+ `type_ids`：输入为成对句子（`pair=`参数）时，记录前后两个句子的token归属
+ `word_ids`：每个token属于哪个word的索引，在分词模型为subword时起作用
+ `words`，同`word_ids`，这个属性后面会被取消

In [16]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

In [17]:
type(tokenizer)

tokenizers.Tokenizer

In [23]:
print('tokenizer.normalizer:     ', tokenizer.normalizer)
print('tokenizer.pre_tokenizer:  ', tokenizer.pre_tokenizer)
print('tokenizer.model:          ', tokenizer.model)
print('tokenizer.post_processor: ', tokenizer.post_processor)
print('tokenizer.decoder:        ', tokenizer.decoder)

tokenizer.normalizer:      <tokenizers.normalizers.BertNormalizer object at 0x000002173CAC4D70>
tokenizer.pre_tokenizer:   <tokenizers.pre_tokenizers.BertPreTokenizer object at 0x000002173CAC4CB0>
tokenizer.model:           <tokenizers.models.WordPiece object at 0x000002173CB36930>
tokenizer.post_processor:  <tokenizers.processors.TemplateProcessing object at 0x000002173CA7C960>
tokenizer.decoder:         <tokenizers.decoders.WordPiece object at 0x000002173CA7CA50>


In [37]:
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
pair = "The second sentence."
encoding = tokenizer.encode(example, pair=pair)
type(encoding)

tokenizers.Encoding

In [38]:
print('encoding.ids: ', encoding.ids)

encoding.ids:  [101, 2026, 2171, 2003, 25353, 22144, 2378, 1998, 1045, 2147, 2012, 17662, 2227, 1999, 6613, 1012, 102, 1996, 2117, 6251, 1012, 102]


In [39]:
print('encoding.tokens: ', encoding.tokens)

encoding.tokens:  ['[CLS]', 'my', 'name', 'is', 'sy', '##lva', '##in', 'and', 'i', 'work', 'at', 'hugging', 'face', 'in', 'brooklyn', '.', '[SEP]', 'the', 'second', 'sentence', '.', '[SEP]']


In [40]:
print('encoding.attention_mask: ', encoding.attention_mask)

encoding.attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [41]:
print('encoding.type_ids: ', encoding.type_ids)

encoding.type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [42]:
print('encoding.word_ids: ', encoding.word_ids)
# print('encoding.words: ', encoding.words)

encoding.word_ids:  [None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None, 0, 1, 2, 3, None]


### 分词器的两类实现

huggingface官方在两个地方提供了分词类：
+ `tokenizer` 包，这里提供的分词类**底层是用Rust实现**的，分词速度非常快，这里实现的分词类被称为 fast-tokenizer.
+ `transformer` 包里也提供了**纯python实现**的分词类，但是该分词类的效率不高，在处理大规模语料时，速度很慢.  

因此，`transformer`包里每个模型的分词类都有两种实现，一个是包内自带的纯python实现的分词类，另一个就是依赖于`tokenizer`包的fast-tokenizer。

有关这两类分词的对比可以参考如下官方文档：
+ [Course --> 6. The tokenizers library -> Fast tokenizers' special powers](https://huggingface.co/course/chapter6/3?fw=pt#fast-tokenizers-special-powers)
+ [Course --> 6. The tokenizers library -> Fast tokenizers in the QA pipeline](https://huggingface.co/course/chapter6/3b?fw=pt#fast-tokenizers-in-the-qa-pipeline)

这两类分词器的实现有一些区别，fast-tokenizer分词类提供的功能多一些，能够进行更加细致的处理。   

比如上面返回的`Encoding`对象里的`word_ids`属性，就记录了分成subword之后，每个subword对应的原有完整词的索引：`[3,3,3]` 对应于`['sy', '##lva', '##in']`。

### 训练tokenizer

这部分主要参考官方 Course 教程的如下几部分：
+ [Course --> 6. The tokenizers library -> Training a new tokenizer from an old one](https://huggingface.co/course/chapter6/2?fw=pt)，从已有的tokenizer进行迁移训练。
+ [Course --> 6. The tokenizers library -> Building a tokenizer, block by block](https://huggingface.co/course/chapter6/8?fw=pt)，介绍了如何训练一个自己的tokenizer。

In [13]:
from datasets import load_dataset_builder, load_dataset
from tokenizers import Tokenizer, normalizers, pre_tokenizers, models, processors, trainers

In [11]:
cd ..

C:\Users\Drivi\Python-Projects\DataAnalysis


In [67]:
path = r'.\datasets\huggingface\wikitext.py'
builder = load_dataset_builder(path, name='wikitext-2-raw-v1')
data_train = load_dataset(path, 'wikitext-2-raw-v1', split='train')
data_train

Found cached dataset wikitext (C:/Users/Drivi/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


Dataset({
    features: ['text'],
    num_rows: 36718
})

In [68]:
# 将数据集组织成生成器，每次返回一个batch的数据
def get_training_corpus_batch(text_data):
    for i in range(0, len(text_data), 1000):
        yield text_data[i: i + 1000]["text"]

In [69]:
# 1. 实例化一个 Tokenizer 类，使用的sub-word分词模型是 WordPiece
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [70]:
# 2. 规范化（Normalization）步骤：分词，去除大小写，词形还原等
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
# 或者可以手动拼凑更加细节的控制
# tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])

In [71]:
# 3. 预处理（pre-tokenization）步骤：
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
# 或者手动进行精细化处理
# tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()])

In [72]:
# 4. 配置sub-word分词模型的训练器
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [74]:
# 5. 使用语料进行训练
tokenizer.train_from_iterator(get_training_corpus_batch(data_train), trainer=trainer)

In [75]:
# 6. 后处理（post-processing）流程：也就是在句子开头加上 [CLS]，句子中间和末尾加上 [SEP]
# 首先获取 [CLS] 和 [SEP] 的 token_id
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
# 然后增加后处理流程
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [76]:
# 7. 保存训练的 tokenizer
tokenizer.save(r".\datasets\huggingface\wikitext-2-raw-v1_tokenizer.json")

+ 读取已训练的分词器，测试一下

In [14]:
tokenizer = Tokenizer.from_file(r".\datasets\huggingface\wikitext-2-raw-v1_tokenizer.json")

In [15]:
single_sen = "Let's test my pre-tokenizer."
pair_sen = "Let's test this tokenizer...", "on a pair of sentences."

single_encoding = tokenizer.encode(single_sen)
pair_encoding = tokenizer.encode(*pair_sen)

print(single_encoding.tokens)
print(single_encoding.type_ids)
print(single_encoding.attention_mask)
print('------------------------------------')
print(pair_encoding.tokens)
print(pair_encoding.type_ids)
print(pair_encoding.attention_mask)

['[CLS]', 'let', "'", 's', 'test', 'my', 'pre', '-', 'tok', '##eni', '##zer', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
------------------------------------
['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '.', '.', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


---

## Transformer

参考：
+ 官方网站 [huggingface-transformer](https://huggingface.co/docs/transformers/index)
+ github地址 [huggingface/transformers](https://github.com/huggingface/transformers)

`transformer`包基于transformer结构，实现了针对如下任务的各种SOTA模型:
+ **NLP**：包括BERT, DistilBERT, GPT2 等等，可以用于文本分类，NER，问答对，机器翻译等
+ CV：计算机视觉相关的模型
+ Audio：语音处理的模型
+ Multimodal：多模态的模型

### 基本框架

有关transformer包的设计理念可以参考官方文档 [Conceptual Guides -> Philosophy](https://huggingface.co/docs/transformers/philosophy).

transformer 包为了保持简洁，尽量减少了抽象接口，每个模型只有3个必要的抽象部分，这里**以NLP任务**为例：

1. [Configuration](https://huggingface.co/transformers/main_classes/configuration.html)  
基础的类是 `PretrainedConfig`（好像也只有这一个类），用于封装各个模型所需要的参数配置，同时也可以方便的 **导出/导入** 参数配置。  
不同的模型有自己的配置类，但是都是继承于此类。

2. [Models](https://huggingface.co/transformers/main_classes/model.html)  
基础的类有如下三个，用于构建模型：
   + `PreTrainedModel`，pytorch实现的模型都继承了这个类，继承于 `torch.nn.Module` 类.
   + `TFPreTrainedModel`，Tensorflow2.0实现的模型都继承了这个类，继承于 `tf.keras.Model` 类.
   + `FlaxPreTrainedModel`.    
.

3. 预处理类.   
对于NLP任务来说是 [Tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html)，包含两个基础的类，用于将文本特征转换成transformer能处理的序列特征：
   + `PreTrainedTokenizer`，这个是纯Python实现，速度比较慢
   + `PreTrainedTokenizerFast`，这个是基于Rust实现，速度比较快，依赖于上面的`tokenizer`包      
   
   对于视频或音频任务来说，是feature extrator类。
   

上面这三个基础模块，都有如下两个方法用于**导入/导出**对应的配置：
+ `.from_pretrained()`，导入预训练模型的 config/model/tokenizer
  + `pretrained_model_name_or_path`，指定预训练模型的名称或者本地路径
  + `cache_dir`，指定模型的缓存位置
  + `force_download`，bool，指定是否强制下载模型
  + `local_files_only`，bool，指定是否只使用本地模型
  + `mirror`，指定下载的镜像地址

> **`PretrainedConfig`类及其子类，可以直接用于实例化各种模型，但是不能用于实例化预处理类（比如Tokenizer）**.

+ `.save_pretrained()`，导出训练好模型的 config/model/tokenizer

**第一次使用这些语句的时候，如果本地没有对应的模型，会下载这些模型**。
  
在上面的3个基础子模块之上，还提供了如下几个子模块工具：

+ [Pipepline](https://huggingface.co/transformers/main_classes/pipelines.html)  
用于快速使用模型，是对上面三个类的封装
+ [Trainer](https://huggingface.co/transformers/main_classes/trainer.html)  
用于快速训练或者 fine-tune 模型


transformer源码中，所有的模型结构都存放在 `src/transformers/models`文件夹下，每个模型对应于一个文件夹（比如BERT对应的就是`bert`文件夹），每个模型的文件夹内，主要有如下几个`.py`文件，其中的 `xxx` 是对应模型的名称：
+ **`configuration_xxx.py`——KEY**，编写了该模型对应的`PretrainedConfig`子类，比如BERT就是`BertConfig`。
+ **`modeling_xxx.py`——KEY**，存放了pytorch实现的模型结构，要用到的模型类都放在这里面
+ `modeling_tf_xxx.py`，存放tensorflow实现的模型结构
+ `modeling_flax_xxx.py`
+ **`tokenization_xxx.py`——KEY**，用于 分词的 实现类（纯python实现），比如BERT就是`BertTokenizer`类
+ `tokenization_xxx_fast.py`，快速分词的实现类，依赖于`tokenizer`包
+ `convert_*.py`，用于将其它配置文件转换成对应的模型。

----

### Configuration

----------------

### Tokenizer

有关 tokenizer 的使用，可以参考如下官方教程
+ [Tutorials --> Preprocess -> NLP](https://huggingface.co/docs/transformers/preprocessing#nlp)
+ [Conceptual Guide --> Glossary -> Model inputs](https://huggingface.co/docs/transformers/glossary#model-inputs)
+ 接口文档[Transformer-Tokenizer](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/tokenizer)

`Tokenizer`类是处理transformer结构输入的主要类，它用于将文本类的数据转换成transformer结构能够接受的序列数据.  

所有的模型里，Tokenizer 都有两种实现：
  1. 基于python代码的实现，基类为`PreTrainedTokenizer`，在`transformer`包里用python实现的
  2. 基于Rust Library的**快速**实现，基类为`PreTrainedTokenizerFast`，**这些快速分词类的实现依赖于`tokenizer`包**


上述两个类又都是基于`PreTrainedTokenizerBase`类，该类实现了分词处理的主要方法，具体功能如下所示：
  + 对文本进行分词，并将分词后的word映射到对应词典的id
  + 训练分词模型，比如BERT使用的WordPiece分词法
  + 生成特殊的Tokens，比如mask等
  + 对序列进行截断，填充等特殊操作

#### 实例化Tokenizer

`PreTrainedTokenizer`类虽然有实例化参数，但是通常会**使用`.from_pretrained()`方法从文件中读取配置，生成对应的Tokenizer对象**。  

注意，**Tokenizer类不能使用`PreTrainedConfig`类及其子类来实例化**。

通常该对象会有如下属性：
+ `name_or_path`：读取的配置路径
+ `vocab_file_names`：字典文件名
+ `vocab_size`：单词字典的大小
+ `vocab`：单词字典，是一个OrderedDict
+ `ids_to_tokens`：id和token的映射字典，也是一个OrderedDict
+ `wordpiece_tokenizer`：底层使用的分词器
+ `max_model_input_sizes`
+ `max_len_single_sentence`
+ `model_max_length`
+ **`is_fast`：是否为快速分词类**

此外，还有一些特殊token的字符和id，比如：
+ `pad_token`, `pad_token_id`
+ `sep_token`, `sep_token_id`
+ `cls_token`, `cls_token_id`

#### `PreTrainedTokenizer`

该类是所有**slow tokenizer**的基类，有如下常用的方法：

##### `__call__()`

参数如下：
+ `text`，输入的句子或者句子序列，可接受的输入类型有：
  + `str`，单个句子
  + `List[str]`，多个句子。注意，如果这里是 list of word 的形式，那么需要设置`is_split_into_words=True`。
  + `List[List[str]]`，一个batch的句子
+ `text_pair`，**可选**，可接受的输入类型同`text`，**这里传入的句子会和 `text` 里的句子组成一对**，常用于QA或者翻译任务中。
+ `text_target`：这个是用于翻译之类的任务中，`text`对应的目标序列
+ `text_pair_target`
+ `max_length`：设置可接受的最大句子长度
+ `padding`，补全策略，有如下几种输入：
  + `True` or `longest`，使用padding，此时按照最长的句子进行padding
  + `max_length`，按照初始化指定的`max_length`参数或者模型的`model_max_length`填充
  + `False` 或者 `do_not_pad`，不做padding，这是**默认值**。此时返回的数据中，一个batch中的各个句子会有不同的长度。
+ `truncation`，指定是否截断
+ `stride`
+ `return_tensors`：返回值的类型
  + `tf`：返回`tf.constant`对象
  + `pt`：返回`torch.Tensor`对象
  + `np`：返回`numpy.ndarray`对象


`__call__()`方法返回的是一个`BatchEncoding`类对象，封装了返回的结果。

##### `tokenize()`
将文本转换成分词后的 tokens，返回的是 `List[str]`，表示的是文本对应 **分词后** 的 tokens .  
注意，**这个不能处理句子对，只能处理单个句子**！

  
##### `encode()`
将文本转换成 sequences of ids.
+ 它接受的参数和 `__call__()` 方法一样
+ 返回的是 `List[int]`，表示的是文本分词后 tokens 对应的 tokenized ids.


##### `decode()`
用于将分词后的 tokens id 转成分词后的文本，并且去除其中的特殊token.


##### 其他方法

+ `convert_ids_to_tokens()`
+ `conver_tokens_to_ids()`
+ `conver_tokens_to_string()`

#### `PreTrainedTokenizerFast`

快速分词的实现基类，它依赖于`tokenizer`包。

相比与`PreTrainedTokenizer`类，它的初始化方法里，多了两个参数：
+ `tokenizer_object`：是一个`tokenizer.Tokenizer`对象，它是快速分词底层使用的分词器。   
  **如果使用自己训练的快速tokenizer时，就需要使用这个参数来实例化一个transformer使用的分词对象**。
+ `tokenizer_file`：指定`tokenizer.Tokenizer`保存的json对象

它提供的接口和`PreTrainedTokenizer`类似，多了如下的两个方法：

+ `batch_decode()`
+ `train_new_from_iterator()`

**注意，`__call__()`方法返回的仍然是`BatchEncoding`对象，而不是`Tokenizer`类返回的`Encoding`对象**

#### BatchEncoding对象

它有如下几个重要属性：
+ `data`：一个dict，存放分词后的数据，有如下重要的key:
  + `input_ids`，分词后每个 token 对应 vocabulary 的 index —— 这个被**作为输入模型的数据**.
  + `attention_mask`，表明填充的 token 位置，**填充位的 token 对应于 0**.  
  因为一个batch中不同的句子长度不一样，短一些的句子会被padding成同样的长度，此时这些padding的位置需要被记录下来，用于告知模型这部分只是填充使用，不需要参与self-attention。
  + `token_type_ids`：**可选**，通常用于**问答对**的句子中，它也是一个{0,1}掩码的形式，0表示属于问题的句子，1表示属于答案的句子
  + `labels`：**可选**，翻译任务中会用到
  
+ `encoding`：`tokenizer.Encoding`类，对于fast-tokenizer来说，会存放一些特殊的信息
+ `is_fast`：是否为fast-tokenizer的结果
  
  
  
常用的一些方法如下：
+ `to(device)`
+ `sequence_ids(batch_index=0)`：返回指定batch里，句子所属部分的指示值：
  + 对于 `[CLS]`, `[SEP]` 这几个特殊的 token，返回 `None`
  + 对于属于第一个句子的 token，返回 0
  + 对于属于第二个句子的 token，返回 1   

  主要用于成对句子

+ `tokens(batch_index=0)`：返回指定batch里，句子进行分词后的token
+ `words(batch_index=0)`：返回句子中每个token的位置索引
+ `word_ids(batch_index=0)`：这个和`words()`一样，后续会被取代。

> token 和 word 区别在于，使用wordPiece之类的分词模型时，对于一些特殊的词，比如 `Titan RTX`，划分得到的token是: `'titan', 'rt', '##x'`，但是原始的word是 `'titan', 'rtx'`，此时的word_ids 就是用来表示 token 和 原来的 word 之间的对应关系：`[0, 1, 1]`，两个 1 表示 `rt`, `##x` 对应的是原来的一个 word。

+ `token_to_word()`：指定位置的 token 对应于原始文本中的word索引

+ `word_to_tokens()`：指定位置的 word 对应的 token 的起止索引，因为一个word可能被分成多个token，所以是复数

+ `token_to_sequence()`：指定位置的 token 属于sequence中的哪个部分，只返回 {0, 1}

+ `token_to_chars()`

+ `word_to_chars()`

+ `char_to_token()`

+ `char_to_word()`

#### 使用示例

In [1]:
cd ..

D:\Project-Workspace\Python-Projects\DataAnalysis


In [2]:
pwd

'D:\\Project-Workspace\\Python-Projects\\DataAnalysis'

In [3]:
# 导入 BERT tokenizer 的设置
from transformers import BertTokenizer, BertTokenizerFast
model_path = r"bert-pretrained-models\bert-base-uncased" # windows
# model_path = r"bert-pretrained-models/bert-base-uncased" # mac
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_path, local_files_only=True)
tokenizer_fast = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path=model_path, local_files_only=True)

+ 需要注意，这里的tokenizer类型不是`tokenizer.Tokenizer`类，只有fast里的`.backend_tokenizer`是

In [9]:
print('tokenizer.__class__:       ', tokenizer.__class__)
print('tokenizer.basic_tokenizer: ', tokenizer.basic_tokenizer)
print('tokenizer_fast.__class__:  ', tokenizer_fast.__class__)

# 快速版本没有 basic_tokenizer 属性, 只有 backend_tokenizer
# print('tokenizer_fast.basic_tokenizer: ', tokenizer_fast.basic_tokenizer)
print('tokenizer_fast.backend_tokenizer: ', tokenizer_fast.backend_tokenizer)

tokenizer.__class__:        <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
tokenizer.basic_tokenizer:  <transformers.models.bert.tokenization_bert.BasicTokenizer object at 0x0000019C0F00CEB0>
tokenizer_fast.__class__:   <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
tokenizer_fast.backend_tokenizer:  <tokenizers.Tokenizer object at 0x0000019C0D78B090>


In [10]:
# 查看分词表大小
print("tokenizer.vocab_size: ", tokenizer.vocab_size)
print("tokenizer_fast.vocab_size: ", tokenizer_fast.vocab_size)

# 查看分词器所使用的的词表， 这个打印会很长
# tokenizer.vocab

tokenizer.vocab_size:  30522
tokenizer_fast.vocab_size:  30522


In [11]:
# 查看特殊的 tokens 和对应的 id
print(tokenizer.all_special_tokens, ':', tokenizer.all_special_ids)
print(tokenizer.cls_token, ': ',tokenizer.cls_token_id)
print(tokenizer.mask_token, ': ', tokenizer.mask_token_id)
print(tokenizer.pad_token, ': ', tokenizer.pad_token_id)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] : [100, 102, 0, 101, 103]
[CLS] :  101
[MASK] :  103
[PAD] :  0


In [12]:
print(tokenizer_fast.all_special_tokens, ':', tokenizer_fast.all_special_ids)
print(tokenizer_fast.cls_token, ': ',tokenizer_fast.cls_token_id)
print(tokenizer_fast.mask_token, ': ', tokenizer_fast.mask_token_id)
print(tokenizer_fast.pad_token, ': ', tokenizer_fast.pad_token_id)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] : [100, 102, 0, 101, 103]
[CLS] :  101
[MASK] :  103
[PAD] :  0


+ 对单个句子或者一个batch的句子进行分词  
此时所有句子分词后得到的`token_type_ids` 都是0，因为它们属于同一个句子。

In [14]:
sentence = "A Titan RTX has 24GB of VRAM"

# 查看分词后的 tokens
tokenized_words = tokenizer.tokenize(sentence)
print("tokenized_words: ", tokenized_words)

# 查看分词后的 tokens id
tokenized_ids = tokenizer.encode(sentence)
print("tokenized_ids:   ", tokenized_ids)

# 从 token id 映射回 token
decoded_tokens = tokenizer.decode(tokenized_ids)
print("decoded_tokens:  ", decoded_tokens)

tokenized_words:  ['a', 'titan', 'rt', '##x', 'has', '24', '##gb', 'of', 'vr', '##am']
tokenized_ids:    [101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
decoded_tokens:   [CLS] a titan rtx has 24gb of vram [SEP]


In [15]:
# 直接调用 __call__() 方法，可以得到分词后的所有信息，一步到位
batch_encoding = tokenizer(sentence)
print('batch_encoding.__class__: ', batch_encoding.__class__)
print("input_ids:      ", batch_encoding['input_ids'])
print("attention_mask: ", batch_encoding['attention_mask'])
print("token_type_ids: ", batch_encoding['token_type_ids'])

batch_encoding.__class__:  <class 'transformers.tokenization_utils_base.BatchEncoding'>
input_ids:       [101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
token_type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
# 使用 fast 版本分词
batch_encoding = tokenizer_fast(sentence)
print('batch_encoding.__class__: ', batch_encoding.__class__)
print("input_ids:      ", batch_encoding['input_ids'])
print("attention_mask: ", batch_encoding['attention_mask'])
print("token_type_ids: ", batch_encoding['token_type_ids'])

batch_encoding.__class__:  <class 'transformers.tokenization_utils_base.BatchEncoding'>
input_ids:       [101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
token_type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


+ 多个长度不一样的句子要使用 padding，并且由 attention_mask 区分padding的词和正常词

In [18]:
s1 = "this is a short sentence"
s2 = "This is a rather longer sequence. It is at least longer than the sequence 1"

batch_encoding = tokenizer([s1, s2], padding=True)

print("s1.length: ", len(batch_encoding['input_ids'][0]), "; s2.length: ", len(batch_encoding['input_ids'][1]))
print("s1['input_ids']:      ", batch_encoding['input_ids'][0])
print("s2['input_ids']:      ", batch_encoding['input_ids'][1])
print("s1['attention_mask']: ", batch_encoding['attention_mask'][0])
print("s2['attention_mask']: ", batch_encoding['attention_mask'][1])

s1.length:  18 ; s2.length:  18
s1['input_ids']:       [101, 2023, 2003, 1037, 2460, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s2['input_ids']:       [101, 2023, 2003, 1037, 2738, 2936, 5537, 1012, 2009, 2003, 2012, 2560, 2936, 2084, 1996, 5537, 1015, 102]
s1['attention_mask']:  [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s2['attention_mask']:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


+ 对**成对句子**进行分词  
此时`toke_type_ids`就起作用了。

In [23]:
s1 = "this is a short sentence"
s2 = "This is a rather longer sequence"

# 注意，此时传入的方式，不是 list，这两个句子作为一个样本，第一个是 text, 第二个是 text_pair
batch_encoding = tokenizer(text=s1, text_pair=s2, padding=True)
print('input_ids : ', batch_encoding['input_ids'])
print('token_type_ids : ', batch_encoding['token_type_ids'])
print('attention_mask : ', batch_encoding['attention_mask'])

input_ids :  [101, 2023, 2003, 1037, 2460, 6251, 102, 2023, 2003, 1037, 2738, 2936, 5537, 102]
token_type_ids :  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
attention_mask :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [21]:
tokenizer.decode(batch_encoding['input_ids'])

'[CLS] this is a short sentence [SEP] this is a rather longer sequence [SEP]'

In [25]:
# 传入一个batch的句子对
text_batch = [s1, s1]
text_pair_batch = [s2, s2]

batch_encoding = tokenizer(text=text_batch, text_pair=text_pair_batch, padding=True)

print('input_ids : ', batch_encoding['input_ids'])
print('token_type_ids : ', batch_encoding['token_type_ids'])
print('attention_mask : ', batch_encoding['attention_mask'])

input_ids :  [[101, 2023, 2003, 1037, 2460, 6251, 102, 2023, 2003, 1037, 2738, 2936, 5537, 102], [101, 2023, 2003, 1037, 2460, 6251, 102, 2023, 2003, 1037, 2738, 2936, 5537, 102]]
token_type_ids :  [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]
attention_mask :  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


+ `BatchEncoding`对象的常用方法

In [26]:
s1 = "A Titan RTX has 24GB of VRAM"
s2 = "This is a rather longer sequence than sequence s1"
batch_encoding = tokenizer_fast(text=s1, text_pair=s2, padding=True)
print(batch_encoding.__class__)

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [27]:
# 查看指定批次的句子的 前后关系，0 表示属于第一个句子，1表示属于第二个句子
print(batch_encoding.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [28]:
# 查看指定批次的句子分词后的token
# 这里只有一个样本，所以传入的batch_index只能是0
print(batch_encoding.tokens(0))

['[CLS]', 'a', 'titan', 'rt', '##x', 'has', '24', '##gb', 'of', 'vr', '##am', '[SEP]', 'this', 'is', 'a', 'rather', 'longer', 'sequence', 'than', 'sequence', 's', '##1', '[SEP]']


In [30]:
# word_ids 表示的是上面划分的token，对应与原来文本中的哪个词的index，比如 2, 2 表示上面的 rt, ##x 在原始文本中是同一个word。
batch_encoding.word_ids(0)
# 下面的会被取代
# batch_encoding.words(0)

[None, 0, 1, 2, 2, 3, 4, 4, 5, 6, 6, None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, None]

In [37]:
# 这里只有 1 个记录，所以传入的被当做 token_index，返回的是对应index的token，对应于原始文本中的word的index
batch_encoding.token_to_word(batch_or_token_index=3)

2

In [44]:
# 给出 第 2 个位置的word: RTX 对应的 token 起止位置
batch_encoding.word_to_tokens(2)

TokenSpan(start=3, end=5)

In [52]:
# 返回第 n 个 位置的 token，属于哪个序列
print(batch_encoding.token_to_sequence(5))
print(batch_encoding.token_to_sequence(15))

0
1


In [56]:
batch_encoding.token_to_chars(5)

CharSpan(start=12, end=15)

-----------------

### Model

transformer的Model基类`PreTrainedModel`有如下的一些属性或方法：
+ `.base_model`，返回当前使用的模型，是`torch.nn.Module`类对象
+ `get_input_embeddings()`，返回Embedding层，也是一个`nn.Module`类对象
+ `get_output_embeddings()`，返回输出层的 embeddings。


+ 可用的模型列表见官网 [Pretrained models](https://huggingface.co/transformers/pretrained_models.html).
  + BERT模型（名称）有：
    + bert-base-uncased
    + bert-base-cased
    + bert-large-uncased
    + bert-large-cased
    + bert-base-chinese  
.    

  + Distil-Bert模型（名称）有：
    + distilbert-base-uncased
    + distilbert-base-cased
  
  这些模型都比较大，可以提前下载下来。

我一般常用的两个模型是 **BERT** 和 **Distil-BERT**，所以下面会着重关注这两个模型里的实现。

### BERT
[BERT Models](https://huggingface.co/transformers/model_doc/bert.html) 包含如下类（以pytorch为例）
+ `BertConfig`，配置类.  
位于`configuration_bert.py`文件中，该文件中也只有这一个配置类
+ `BertTokenizer`，实现 `WordPiece` 分词的类.  
位于`tokenization_bert.py`文件中，该文件中还有 `BasicTokenizer` 和 `WordpieceTokenizer` 两个类，不过`BertTokenizer`会调用它们.
+ `BertForPreTrainingOutput`，记录BERT模型的输出.
+ `BertModel`，最基本的 BERT 模型类，它返回的是 BERT 模型的原始结果，包括隐藏层状态.
+ `BertForPreTraining`，在 `BertModel`的输出上加了一层处理，以下的几个模型都是如此.
+ `BertForMaskedLM`
+ `BertForNextSentencePrediction`
+ `BertForSequenceClassification`
+ 还有其他的一些基础工具类和其他任务对于的BERT模型类，这里就不介绍了。   

注意：
> huaggingface-transformer 里，**并没有使用pytorch内部提供的nn.Transformer相关的模型，而是自己重新写了一套实现逻辑**.

#### BertConfig

官方文档 [BertConfig](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertConfig).   
封装了Bert的配置

#### 底层实现类

huggingface-transformer的实现里，除了上述几个可以直接用于任务的模型类之外，还有一些底层的实现类没有对外暴露，这些实现类是上述模型的基础。

<img src="images/transformer-bert.png" width="50%" align="center">    


+ `BertEmbeddings`：
  + 用于将输入的 `input_ids`，`token_type_ids`，`position_ids` 转成词向量，并进行相加（原论文中是这么做的）  
  .


+ **`BertSelfAttention`-KEY**：
  + 实现transformer的 self-attention 操作，也就是 ${\rm softmax}(\frac{ {\rm query} \cdot {\rm key}^T}{\sqrt{d_k}}) \cdot {\rm value}$ 这一部分.  
  + 这里的self-attention操作，既可以是 encoder 里的 self-attention，也可以是 decoder 里的 self-attention
  + 如果是 decoder 里的 self-attention，又分为两种情况：
    + decoder的 Masked-self-attention（第一层），此时 q,k,v 都来自于输出序列的同一个
    + decoder的 cross-self-attention（第二层），此时 q,k 来自于 encoder 的输出，v 来自于上一层的 encoder
  + 为了兼容上述3种情况，它的`forward`方法里，可以传入的参数很多，用于控制具体实现哪里的self-attention.
  + **输出最多为 长度=3 的 tuple**：`(context_layer, attention_probs, past_key_value)`
    + `context_layer` 是 self-attention 最后的计算结果
    + `attention_probs` 是 ${\rm softmax}(\frac{ {\rm query} \cdot {\rm key}^T}{\sqrt{d_k}})$ 的结果
    + `past_key_value` 是一个 长度=2 的 tuple：`(key, value)`，存放的是 decoder 中第二层的 cross-self-attention 需要传入的 key,value.    
  .


+ `BertSelfOutput`：实现transformer里的 self-attention 之后的 **Add & Norm** 操作
  
+ `BertAttention`：封装了 `BertSelfAttention` + `BertSelfOutput`，形成一个 **Attention** 层
  + 输出：和 `BertSelfAttention` 相同，只是对 `context_layer` 进行了 Add & Norm 处理，其他保持不变   
  .


+ `BertIntermediate`: 内部是一个 linear 层 + 激活函数，也就是 Attention 层后面的 **FeedForward** 全连接层
+ `BertOutput`：这个和 `BertSelfOutput` 没有区别，只是为了实现 FeedForward 层后面的 **Add & Norm** 操作


+ **`BertLayer`-KEY**：封装了 `BertAttention` + `BertAttention`(可选) +  `BertIntermediate` + `BertOutput`，构成 encoder 或者 decoder 里的一层.    
  它提供了如下两个参数：
  + `config.is_decoder`：为 True 时，作为 decoder 使用，输出 decoder 的结果，否则作为 encoder 使用，输出 encoder 的隐藏层状态
  + `config.add_cross_attention`：为 True 时，会再添加一个 `BertAttention` 层，也就是 decoder 内部中间的 self-attention 层，称为 **cross-attention**   
  
  **输出：长度=4 的tuple**: `(context_layer, attention_probs, cross_attention_probs, past_key_value)`
  + `context_layer` 是 输出的状态向量，它可能经过了两个 `BertAttention` 层
  + `attention_probs` 是第一个 `BertAttentnion` 层的输出
  + `cross_attention_probs` 是第二个 `BertAttentnion`(cross-attention) 层输出的
  + `past_key_value` 现在可能是一个 长度=4 的tuple: `(key, value, cross_attention_key, cross_attention_value)`，前两个是第一个self-attention的输出，后两个是 cross-attention的输出.    
  .


+ `BertEncoder`：内部是多个作为 encoder 使用的 `BertLayer` 层   
**输出**是一个 `BaseModelOutputWithPastAndCrossAttentions` 类，里面封装了如下结果：
  + `last_hidden_state`：最后一个 encoder 的输出，也就是最后的 `context_layer`
  + `hidden_states`：以 tuple 的形式记录了所有 encoder 的输出
  + `attentions`：以 tuple 的形式记录了所有 encoder 的 `attention_probs`
  + `cross_attentions`: 以 tuple 的形式记录了所有decoder的 `cross_attention_probs`
  + `past_key_values`：以 tuple 的形式记录了所有decoder的 `past_key_value`

#### 输出结构

除了上述属于 transformer 架构的组件类，还提供了一些专门用于 BERT 架构的输出组件类。

+ **`BertPooler`-KEY**：内部是一个 Linear + tanh激活函数，也就是一个全连接层，但是它**只会处理 `BertLayer` 输出的第一个 token 的隐藏层状态**。   
对应 BERT 的 Masked-Language-Model 训练方式里，第一个 token `[CLS]` 的输出，**它不会改变隐藏层状态向量的维度**.

+ `BertPredictionHeadTransform`：内部是一个 全连接层(Linear+激活函数) + LayerNorm，它会处理 `BertLayer` 输出的所有序列的隐藏层状态，相当于对 `BertLayer` 或者 `BertEncoder` 的输出做一次 FeedForward 处理，它**不会改变隐藏层状态向量的维度**。

+ **`BertLMPredictionHead`-KEY**：内部是一个 `BertPredictionHeadTransform` + Linear，其中线性变换会改变隐藏层向量维度：`hidden_size` --> `vocab_size`。  
名称中的"LM"应该指的是Language Model，也就是说这一层是为了用于语言模型对transformer-encoder的输出进行的变换。

+ `BertOnlyMLMHead`：内部就是 `BertLMPredictionHead`，没有加其他的操作。  
专门用于使用 Masked-Language-Model(MLM) 方式训练时获取transformer-encoder的输出。

+ `BertOnlyNSPHead`：内部是一个 Linear变换，但是维度从 `hidden_size` -> 2。   
专门用于使用 Next Sentence Prediction(NSP) 方式训练时将第一个token `[CLS]` 转成一个 表示二分类结果的向量。  
它前面通常会有一个 `BertPooler` 层，用于获取第一个token `[CLS]` 的隐藏层状态向量。

+ `BertPreTrainingHeads`：内部是 `BertLMPredictionHead` + 等价于`BertOnlyNSPHead`的 Linear变换.  
这个应该是用于同时采用 MLM + NSP 方式训练BERT时，同时获取两者的输出.

有了上述的 transformer 基础组件类 和 BERT 组件类的支持，就可以构建下面使用的 BERT 模型了。

#### BertModel

最基本的BERT架构，既可以作为 transformer-encoder 或者 transformer-decoder 使用，输出原始的隐藏层状态。

查看源码时，会发现它的初始化方法里，有3个层：
1. `self.embeddings = BertEmbeddings(config)`  
2. `self.encoder = BertEncoder(config)`  
这一层内部是`self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])`，其中的`BertLayer`
3. `self.pooler = BertPooler(config) if add_pooling_layer else None`


+ 初始化的一些重要参数：
  + `is_decoder`
  + `add_cross_attention`
  + `add_pooling_layer`，是否添加最后的 `BertPooler` 层，默认为True
  
  上述参数中，除了`add_pooling_layer`，其他参数都是封装在 `BertConfig` 类中的。


+ 前向传播方法：  
`forward(input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_values=None, use_cache=None, output_attentions=None, output_hidden_states=None, return_dict=None)`
  + `input_ids`：`shape=(batch_size, sequence_length)`，每个sequence是一个 list of int，其中每个值是该word在词典中的indice，经过embedding层之后，每个word的indice会被转换成word embedding的词向量。
  + `attention_mask`，用于表示该位置的词语是否需要be attended，主要是处理padding的词语，这些padding的词语不需要参与计算。
  + `token_type_ids`，用于区分成对句子中两个句子，为 {0, 1} 掩码
  + `position_ids`，位置编码，可选
  + `head_mask`，
  + `inputs_embeds`，输入的embedding向量，`shape=(batch_size, sequence_length, hidden_size)`，如果不想传入`input_ids`经过内部的`BertEmbedding`转成向量，就可以在这里传入自定义的embedding表示向量，注意，这个**不能和 `input_ids` 同时使用**.
  + `encoder_hidden_states`，作为decoder里的cross-attention使用时，需要的encoder输出就从这个参数传入
  + `encoder_attention_mask`，
  + `past_key_values`，包含了前一个attention block了计算的 key 和 value —— 这个参数的使用有点迷惑。
  + `output_attentions`，是否输出所有attention layer 的结果，默认 False
  + `output_hidden_states`，是否输出所有attention layer的隐藏层状态，默认 False


`forward()`方法的返回值是如下的对象（在`return_dict=True`时）：
+ `BaseModelOutputWithPoolingAndCrossAttentions`类对象，它有如下属性：
  + `last_hidden_state`，shape=`(batch_size, sequence_length, hidden_size)`，最后一层的输出
  + `pooler_output`，shape=`(batch_size, hidden_size)`，最后一层中 `[CLS]` token 对应的输出，经过了 `BertPooler` 的处理
  + `hidden_states`，只有当`output_hidden_states=True`时才会返回.  
    + 返回的是一个 长度= 1 + hidden_layers_num 的tuple, 每个元素对应于一层的隐状态（包括了 Embedding 层），
    + 每层的隐状态 shape=`(batch_size, sequence_length, hidden_size)`
  + `attentions`，只有当`output_attentions=True`时才会返回.  
    + 返回一个 长度=hidden_layers_num 的tuple，对应于 每一层 的attention，也就是 `BertSelfAttention` 输出的 `attention_probs`
    + 每一层的attention.shape=`(batch_size, num_heads, sequence_length, sequence_length)`，
  + `cross_attentions`，只有当`output_attentions=True`时才会返回.  
    + 返回一个 长度=hidden_layers_num 的tuple，对应于 每一层 的 cross-attention，也就是 `BertLayer`里设置`add_cross_attention=True`时输出的`cross_attention_probs`
    + 每一层的attention.shape=`(batch_size, num_heads, sequence_length, sequence_length)`
  + `past_key_values`，只有当`use_cache=True` 时才会返回
  
如果`return_dict=False`，那么返回的就是一个 tuple，其中包含的具体元素要看配置.

In [4]:
from transformers import BertConfig, BertTokenizer, BertModel

In [6]:
cd ..

D:\Project-Workspace\Python-Projects\DataAnalysis


In [7]:
pwd

'D:\\Project-Workspace\\Python-Projects\\DataAnalysis'

In [9]:
# model_name = "./bert-pretrained-models/bert-base-uncased"   # mac
model_name = r".\bert-pretrained-models\bert-base-uncased"    # windows

config = BertConfig.from_pretrained(model_name, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, local_files_only=True)
# 使用 config 对象初始化 BERT模型，但是不能 config 来初始化 tokenizer
model = BertModel(config)
# 或者
# model = BertModel.from_pretrained(model_name, local_files_only=True)


print("config.__class__:    ", config.__class__)
print("tokenizer.__class__: ", tokenizer.__class__)
print("model.__class__:     ", model.__class__)

Some weights of the model checkpoint at .\bert-pretrained-models\bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.__class__:     <class 'transformers.models.bert.configuration_bert.BertConfig'>
tokenizer.__class__:  <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
model.__class__:      <class 'transformers.models.bert.modeling_bert.BertModel'>


In [10]:
# 单个句子
# sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# 两个句子
sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# return_tensors 指定返回的结果为 torch.Tensor，这样就不用做转换了，
# 注意，必须要设置 padding，否则得到的 tensor 维度不一样，无法转成 tensors
sentence_encode = tokenizer(sentence, padding=True, return_tensors='pt')

print(sentence_encode)
print(tokenizer.decode(sentence_encode['input_ids'][0]))
print(tokenizer.decode(sentence_encode['input_ids'][1]))

{'input_ids': tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,   102,     0,
             0,     0,     0,     0],
        [  101,  1996,  2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,
          2314,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS] after stealing money from the bank vault [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] the bank robber was seen fishing on the mississippi river bank. [SEP]


In [11]:
# 查看下 BertConfig 里的配置
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [12]:
# 将加载的预训练模型置于 evaluation 状态，这样会关闭其中的 dropout
model.eval()

# 预测时，使用 .no_grad() 方式会加快计算
with torch.no_grad():
    # 可以直接将 tokenizer 得到的输出作为输入，只要使用拆包技巧就行
    outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True)
    
outputs.__class__

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [14]:
# 两个句子: batch_size=2, 每个句子序列的长度 sequence_length=14, 
# 最后一层的 hidden_size=768 —— 这个由预训练模型的配置决定
outputs.last_hidden_state.shape

torch.Size([2, 14, 768])

In [15]:
# 对应于 [CLS] token 的 embedding，2 个句子，所以返回了两个
outputs.pooler_output.shape

torch.Size([2, 768])

In [115]:
# 查看每一个隐藏层的状态
print(len(outputs.hidden_states))
outputs.hidden_states.__class__

13


tuple

In [96]:
# Embedding 层的 隐状态
outputs.hidden_states[0].shape

torch.Size([2, 14, 768])

In [109]:
# 第一个 self-attention 层的 隐状态
outputs.hidden_states[1].shape

torch.Size([2, 14, 768])

In [116]:
# attention 的信息
print(len(outputs.attentions))
outputs.attentions.__class__

12


tuple

In [117]:
# 第一个 self-attention 层的 atttention shape
# 2 个句子 batch_size=2, 12 个 heads, sequence_length=14, sequence_length=14
outputs.attentions[0].shape

torch.Size([2, 12, 14, 14])

+ 也可以用 切片 的方式，直接从 `outputs` 中获取前两个的值

In [60]:
# t1 对应于 last_hidden_state, t2 对应于 pooler_output
t1, t2 = outputs[:2]
print(t1.__class__)
print(t1.shape)
print(t2.shape)

<class 'torch.Tensor'>
torch.Size([2, 14, 768])
torch.Size([2, 768])


In [48]:
model.eval()
# return_dict=False，那么返回的就是一个 tuple
outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True, return_dict=False)

In [50]:
print("outputs.__class__: ", outputs.__class__)
print("outputs.len:       ", len(outputs))

outputs.__class__:  <class 'tuple'>
outputs.len:        4


In [52]:
outputs[0].shape

torch.Size([2, 14, 768])

#### BertForPreTraining

在 BertModel 的输出后，增加了一层 `BertPreTrainingHeads()`，如果要自己训练一个BERT模型，从这个类出发比较方便.

通过分析源码可以发现，它（并行）做了如下两件事：
1. 调用 `nn.Linear()` 将 BertModel 输出的 `[CLS]` token （也就是pooled_output） 转换成 2 维向量，用于表示二分类的值——用于 Next Sentence 的训练方式
2. 调用 `BertLMPredictionHead()` ，对 BertModel 最后一层的输出 last_hidden_state 进行处理——用于做 MaskLanguageModel 的训练    
内部处理逻辑如下：
   + 调用 `BertPredictionHeadTransform()` 对 last_hidden_state 做 线性变换 + 激活函数 + LayerNormalization，输出的 shape 不变
   + 调用 `nn.Linear()` 将上一步得到的向量从 hidden_size 转换成 vocab_size 维度的向量


+ `forward()`方法，参数相比于 `BertModel.forward()` 多了如下两个：
  + `label`, shape=`(batch_size, sequence_length)`，用于计算 masked language modeling Loss 的标记.   
  其中的取值范围应为 `[-100, 0, ..., config.vocab_size]`，也就是比 `input_ids` 的范围多了一个 -100, -100 表示对应位置的token 会被 mask，后续计算loss时，被标记为 -100 的token会被忽略掉。
  + `next_sentence_label`, shape=`(batch_size,)`，用于计算 next sequence prediction(分类问题) Loss 的标记.   
  取值只有 `{0, 1}`
    + 0 表示 后一句 是连着 前一句
    + 1 表示 后一句 是随机抽取的


+ 返回值是一个封装的 `BertForPreTrainingOutput()` 对象（`return_dict=False`时）
  + `loss`，shape=`(1,)`，保存了 MaskLM 的 Loss 和 next sentence prediction 的 **Loss 之和**——不懂这里为什么要求和。
  + `prediction_logits`，shape=`(batch_size, sequence_length, config.vocab_size)`，记录了softmax之前的score
  + `seq_relationship_logits`, shape=`(batch_size, 2)`，记录了 next sequence prediction 在 softmax 之前的 score
  + `hidden_states`，就是 `BertModel` 的 hidden_states
  + `attentions`，`BertModel`的对应输出

In [8]:
from transformers import BertTokenizer, BertConfig, BertForPreTraining

In [11]:
model_name = "./BERT/bert-pre-trained-models/bert-base-uncased"
# model_name = "./BERT/bert-pre-trained-models/distilbert-base-uncased/"

config = BertConfig.from_pretrained(model_name, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, local_files_only=True)
model = BertForPreTraining.from_pretrained(model_name, local_files_only=True)

print("config.__class__:    ", config.__class__)
print("tokenizer.__class__: ", tokenizer.__class__)
print("model.__class__:     ", model.__class__)

Some weights of BertForPreTraining were not initialized from the model checkpoint at ./BERT/bert-pre-trained-models/bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.__class__:     <class 'transformers.models.bert.configuration_bert.BertConfig'>
tokenizer.__class__:  <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
model.__class__:      <class 'transformers.models.bert.modeling_bert.BertForPreTraining'>


In [19]:
# 两个句子
sentence = ["After stealing money from the bank vault", "the bank robber was seen fishing on the Mississippi river bank."]

# return_tensors 指定返回的结果为 torch.Tensor，这样就不用做转换了，
# 注意，必须要设置 padding，否则得到的 tensor 维度不一样，无法转成 tensors
sentence_encode = tokenizer(sentence, padding=True, return_tensors='pt')
print(sentence_encode)

outputs = model(**sentence_encode, output_hidden_states=True, output_attentions=True)

{'input_ids': tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,   102,     0,
             0,     0,     0,     0],
        [  101,  1996,  2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,
          2314,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
outputs.__class__

transformers.models.bert.modeling_bert.BertForPreTrainingOutput

In [14]:
outputs.loss

In [17]:
outputs.prediction_logits.shape

torch.Size([2, 14, 30522])

In [18]:
outputs.seq_relationship_logits.shape

torch.Size([2, 2])

#### BertForSequenceClassification

这个类专门用于 序列分类，它的内部很简单：
1. `BertModel()` 输出 pooled_output(`[CLS]` token 对应的 hidden_state)
2. `nn.Dropout()` 处理 pooled_output
3. `nn.Linear()` 映射到 多分类的类别 logits


+ `forward()`方法

+ 返回的是`SequenceClassifierOutput()`类对象

---

### Distil-BERT

[DistilBERT Models](https://huggingface.co/transformers/model_doc/distilbert.html) 包含如下类:   

---

## 训练与Fint-Tune

huggingface官方提供了一个使用transformer的 [Course](https://huggingface.co/course/chapter0/1?fw=pt)，这个教程写的很不错。

如果想自己训练一个模型，可以参考其中的如下内容：
+ [Course --> 6. The Tokenizer Library](https://huggingface.co/course/chapter6/1?fw=pt)
  + [Training a new tokenizer from an old one](https://huggingface.co/course/chapter6/2?fw=pt#training-a-new-tokenizer-from-an-old-one)
  + [Building a tokenizer, block by block](https://huggingface.co/course/chapter6/8?fw=pt#building-a-tokenizer-block-by-block)
+ [Course --> 7. Main NLP Task](https://huggingface.co/course/chapter7/1?fw=pt)
  + [Fine-tuning a masked language model](https://huggingface.co/course/chapter7/3?fw=pt#finetuning-a-masked-language-model)
  + [Training a causal language model from scratch](https://huggingface.co/course/chapter7/6?fw=pt#training-a-causal-language-model-from-scratch)
  
  
除了上述两个教程之外，huggingface在`transformer`模块里，还提供了如下类作为辅助工具：
+ `DataCollator`：用于补齐填充数据
+ `Trainer`：封装训练过程，**只适用于PyTorch**框架

----

### DataCollator

官方文档：
+ [API：Data Collator](https://huggingface.co/docs/transformers/main_classes/data_collator)

DataCollator 用于从 list of dataset elements 中生成一个 batch 的数据，对该batch的数据进行如下的一些操作：
+ 对batch中的每个样本序列进行对齐padding操作 —— 注意，**这个序列对齐只针对某个batch，不同batch的序列长度可以是不一样的**
+ 用于 Masked Language Model 时，对输入的序列进行 Mask 处理

#### DefaultDataCollator
默认的collator，**它会检测 dataset 中是否有 `labels` 或者 `label_ids` 这两个特征，有的话，会单独处理**。

DataCollator的输入是 list of dict-like 的内容，输出是一个 dict-like，每个 key 是一个特征，value 是 list，对应于该特征下的 batch。

In [1]:
# 以 DefaultDataCollator 为例
from transformers import DefaultDataCollator
dc = DefaultDataCollator()

data_list = [
    {'label': 1, 'col-1': 10, 'col-2': 20},
    {'label': 2, 'col-1': 11, 'col-2': 21},
    {'label': 3, 'col-1': 12, 'col-2': 22}
]

dc_data = dc(data_list)
dc_data

{'labels': tensor([1, 2, 3]),
 'col-1': tensor([10, 11, 12]),
 'col-2': tensor([20, 21, 22])}

#### DataCollatorWithPadding

用于动态对每个 batch 的样本进行 padding.



#### DataCollatorForLanguageModeling

用于对 Language Model 的输入进行处理，比如对于BERT 的输入会执行 随机Mask 的操作.

注意：
> 如果使用了 WordPiece 之类的分词器，获得 subword 时（比如 `hug`,`##ging` 之类的词），这个类 Mask 的是 subword，不会 Mask 整个word （这里是`hugging`）

#### DataCollatorForWholeWordMask

用于对 Language Model 的输入进行处理，比如对于BERT 的输入会执行 随机Mask 的操作.

即使是使用 Subword 分词器，这个类 Mask 的还是整体的word ，不过要求其中的 subword 必须以指定的符号为前缀（比如`##`）

-----
### Trainer

官方文档：
+ [Transormer --> Main Class --> Trainer](https://huggingface.co/docs/transformers/v4.25.1/en/main_classes/trainer#transformers.Trainer)：API文档
+ [Transformer --> Tutorial --> Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer)：`transformer`包的教程，比较详细，除了介绍`Trainer`的使用外，还有 Keras 和原始 pytorch 的训练代码
+ [Cource --> 3. Fine-Tuning a pretrained model --> Fine-tuning a model with the Trainer API](https://huggingface.co/course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api)：Course里的Trainer教程，可以作为上面教程的补充


Trainer-API的使用主要分为两个部分：
1. 使用`TrainingArguments`类封装所有的超参数，具体参数可以查看该类的API文档 [Trainer --> TrainingArguments](https://huggingface.co/docs/transformers/v4.25.1/en/main_classes/trainer#transformers.TrainingArguments)
2. 实例化`Trainer`类

Trainer-API 主要提供了如下 **两组** 共 **4 个类**：
+ `TrainingArguments` + `Trainer`，用于普通模型的训练
+ `Seq2SeqTrainingArguments` + `Seq2SeqTrainer`：用于序列模型的训练，这两个类分别继承了上面的类，`Seq2SeqTrainingArguments`扩充了几个属性，`Seq2SeqTrainer` 重写了几个父类方法

如果要自定义一些训练流程，可以继承`Trainer`类，然后重写其中的如下方法：
+ `trainning_step()`
+ `compute_loss()`
+ `evaluate()`
+ `prediction_step()`
+ `predict()`

需要注意的是，Trainer-API是专门为`transformer`包里的模型定制的训练流程，所以继承重写上述方法的时候，需要遵循一些约定，否则Trainer-API会出现意想不到的结果


> **稍微看了下`Trainer`的源码，发现其中添加了许多逻辑，比如参数检查，最大batch_size的检查，各种回调方法，多GPU的支持，梯度修剪等等。  
我的感觉是，这个API可以用，但是不要滥用，其中封装了许多对于初学者来说不必要的逻辑。  
不过如果作为研究的对象还不错，其中有许多优化求解的操作，应该值得研究。**
